# I- Importation du fichier production

On commence par importer les bibliothèques

In [17]:
import pandas as pd
import numpy as np
import pickle # pour le chargement du modèle de régression logistique
import joblib # pour le chargement du modèle Random Forest
import tensorflow as tf # pour le chargement du modèle de réseau neuronal
import statsmodels.api as sm # pour l'ajout de la constante

In [18]:
# Lecture du fichier avec , comme séparateur
data = pd.read_csv("data_raw/billets_production.csv", sep=',')

# Affiche les 5 premières lignes
data.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


On extrait 'id' pour ne garder que les colonnes numériques

In [19]:
# Extraire la colonne 'id' pour l'utiliser plus tard
data_ids = data['id']

In [20]:
# Exclure la colonne 'id' avant l'entraînement
X = data.drop(columns=['id'])
# Affiche les premières lignes sans la colonne 'id'
print(X.head())

   diagonal  height_left  height_right  margin_low  margin_up  length
0    171.76       104.01        103.54        5.21       3.30  111.42
1    171.87       104.17        104.13        6.00       3.31  112.09
2    172.00       104.58        104.29        4.99       3.39  111.57
3    172.49       104.55        104.34        4.44       3.03  113.20
4    171.65       103.63        103.56        3.77       3.16  113.33


# II- Chargement du modèle de régression logistique

In [21]:
# Charger le modèle sauvegardé
with open('logistic_model_aic.pkl', 'rb') as f:
    logistic_model_aic = pickle.load(f)

print("Modèle chargé avec succès.")


Modèle chargé avec succès.


On vérifie les colonnes utilisées dans le modèle

In [22]:
# Afficher les colonnes utilisées par le modèle AIC
print(logistic_model_aic.model.exog_names)


['const', 'height_right', 'margin_low', 'margin_up', 'length']


On exclue les colonnes qui ne sont pas présentes dans le modèle

In [23]:
# Exclure toutes les colonnes non pertinentes
X_aic = X[['height_right', 'margin_low', 'margin_up', 'length']]


On ajoute une colonne 'constante'

In [24]:
# Ajouter une colonne de constante (biais) à X
X_with_const = sm.add_constant(X_aic)

# Vérifier les premières lignes de X pour s'assurer que la constante est bien ajoutée
print(X_with_const.head())


   const  height_right  margin_low  margin_up  length
0    1.0        103.54        5.21       3.30  111.42
1    1.0        104.13        6.00       3.31  112.09
2    1.0        104.29        4.99       3.39  111.57
3    1.0        104.34        4.44       3.03  113.20
4    1.0        103.56        3.77       3.16  113.33


## Prédictions du modèle et probabilités

In [25]:
# Faire des prédictions avec le modèle chargé
reg_log_prob = logistic_model_aic.predict(X_with_const)
# Afficher les premières prédictions
print(reg_log_prob[:5])

0    0.000035
1    0.000002
2    0.000005
3    0.995109
4    0.999994
dtype: float64


In [26]:
# Appliquer un seuil pour obtenir les prédictions
threshold = 0.5
reg_log_pred = np.where(reg_log_prob > threshold, "True", "False")

# Afficher les premières prédictions
print(reg_log_pred[:5])


['False' 'False' 'False' 'True' 'True']


In [27]:
data["reg_log_pred"] = reg_log_pred
data["reg_log_prob"] = reg_log_prob * 100
data

,diagonal,height_left,height_right,margin_low,margin_up,length,id,reg_log_pred,reg_log_prob
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,0.003503
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.000171
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.000501
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,99.510899
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,99.999398


# III- Chargement du modèle Random Forest

In [28]:
# Charger le modèle Random Forest AIC
with open('random_forest_aic.pkl', 'rb') as f:
    random_forest_aic = joblib.load(f)

print("Modèle Random Forest chargé avec succès.")


Modèle Random Forest chargé avec succès.


## Prédictions du modèle et probabilités

In [30]:
# Faire des prédictions avec le modèle chargé
rf_pred = random_forest_aic.predict(X_aic)

# Si tu souhaites avoir les probabilités
rf_prob = random_forest_aic.predict_proba(X_aic)[:, 1]

# Afficher les premières prédictions
print(rf_pred[:5])


[False False False  True  True]


In [31]:
data["rf_pred"] = rf_pred
data["rf_prob"] = (rf_prob * 100)
data

,diagonal,height_left,height_right,margin_low,margin_up,length,id,reg_log_pred,reg_log_prob,rf_pred,rf_prob
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,0.003503,False,1.0
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.000171,False,0.0
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.000501,False,0.0
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,99.510899,True,99.0
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,99.999398,True,100.0


# IV- Chargement du modèle de Réseau Neuronal

In [32]:
# Charger le modèle sauvegardé
loaded_model = tf.keras.models.load_model('neural_network_model')

# Vérifier le modèle chargé
loaded_model.summary()  # Afficher un résumé du modèle pour vérifier qu'il est chargé correctement

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


## Prédictions du modèle et probabilités

In [36]:
neurone_prob = loaded_model.predict(X)

print(neurone_prob)

[[1.0704696e-03]
 [1.0218886e-05]
 [7.5531006e-04]
 [8.6388087e-01]
 [9.9949622e-01]]


In [37]:
# Appliquer un seuil pour obtenir les prédictions
threshold = 0.5
neurone_pred = np.where(neurone_prob > threshold, "True", "False")

# Afficher les premières prédictions
print(neurone_pred)

[['False']
 ['False']
 ['False']
 ['True']
 ['True']]


In [38]:
data["neurone_pred"] = neurone_pred
data["neurone_prob"] = (neurone_prob * 100)
data

,diagonal,height_left,height_right,margin_low,margin_up,length,id,reg_log_pred,reg_log_prob,rf_pred,rf_prob,neurone_pred,neurone_prob
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,0.003503,False,1.0,False,0.107047
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.000171,False,0.0,False,0.001022
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.000501,False,0.0,False,0.075531
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,99.510899,True,99.0,True,86.388084
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,99.999398,True,100.0,True,99.949623
